In [1]:
from google.colab import drive
drive.mount('/content/googledrive',force_remount=True)

Mounted at /content/googledrive


In [2]:
%cd /content/googledrive/MyDrive/ECE661_HW4

/content/googledrive/MyDrive/ECE661_HW4


In [3]:
!ls

 data		  net_after_finetune.pt		        quantized_net_after_finetune.pt
 FP_layers.py	  net_after_global_iterative_prune.pt   resnet20.py
'hw4 (1).ipynb'   net_after_iterative_prune.pt	        train_util.py
'hw4 (2).ipynb'   pretrained_model.pt		        Untitled0.ipynb
 LAB1.ipynb	  __pycache__


In [8]:
import torch

### Lab2 (a) Model preperation

In [4]:
from resnet20 import ResNetCIFAR
from train_util import train, finetune, test
import torch
import numpy as np

import time

import torchvision.transforms as transforms
import torchvision
import torch.nn as nn
import torch.optim as optim

from FP_layers import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [6]:
import sys
sys.path.append('/content/googledrive/MyDrive/ECE661_HW4')

In [6]:
net = ResNetCIFAR(num_layers=20, Nbits=None)
net = net.to(device)

In [7]:
# Load the best weight paramters
net.load_state_dict(torch.load("pretrained_model.pt"))
test(net)

Files already downloaded and verified
Test Loss=0.3231, Test accuracy=0.9151


### Lab2 (b) Prune by percentage

In [8]:
def prune_by_percentage(layer, q=70.0):
    """
    Pruning the weight paramters by threshold.
    :param q: pruning percentile. 'q' percent of the least
    significant weight parameters will be pruned.
    """
    # Convert the weight of "layer" to numpy array
    weights=layer.weight.data.cpu().numpy()

    # Compute the q-th percentile of the abs of the converted array
    threshold=np.percentile(np.abs(weights),q)

    # Generate a binary mask same shape as weight to decide which element to prune
    mask=np.abs(weights) >= threshold

    # Convert mask to torch tensor and put on GPU
    mask=torch.from_numpy(mask).float().to(device)

    # Multiply the weight by mask to perform pruning
    layer.weight.data *= mask
    pass


In [25]:
q_values = [30, 50, 70]

for q in q_values:
    print("Pruning with q =", q)
    net.load_state_dict(torch.load("pretrained_model.pt"))

    for name, layer in net.named_modules():
        if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
            # Apply pruning by percentage
            prune_by_percentage(layer, q=q)

            # Count the number of zeros and total parameters for sparsity
            np_weight = layer.weight.data.cpu().numpy()
            zeros = np.sum(np_weight == 0)
            total = np_weight.size

            # Calculate and print sparsity
            sparsity = zeros / total
            print(f'Sparsity of {name}: {sparsity:.2%}')

    # Test the pruned model
    test(net)


Pruning with q = 30
Sparsity of head_conv.0.conv: 30.09%
Sparsity of body_op.0.conv1.0.conv: 29.99%
Sparsity of body_op.0.conv2.0.conv: 29.99%
Sparsity of body_op.1.conv1.0.conv: 29.99%
Sparsity of body_op.1.conv2.0.conv: 29.99%
Sparsity of body_op.2.conv1.0.conv: 29.99%
Sparsity of body_op.2.conv2.0.conv: 29.99%
Sparsity of body_op.3.conv1.0.conv: 30.01%
Sparsity of body_op.3.conv2.0.conv: 30.00%
Sparsity of body_op.4.conv1.0.conv: 30.00%
Sparsity of body_op.4.conv2.0.conv: 30.00%
Sparsity of body_op.5.conv1.0.conv: 30.00%
Sparsity of body_op.5.conv2.0.conv: 30.00%
Sparsity of body_op.6.conv1.0.conv: 30.00%
Sparsity of body_op.6.conv2.0.conv: 30.00%
Sparsity of body_op.7.conv1.0.conv: 30.00%
Sparsity of body_op.7.conv2.0.conv: 30.00%
Sparsity of body_op.8.conv1.0.conv: 30.00%
Sparsity of body_op.8.conv2.0.conv: 30.00%
Sparsity of final_fc.linear: 30.00%
Files already downloaded and verified
Test Loss=0.3698, Test accuracy=0.9028
Pruning with q = 50
Sparsity of head_conv.0.conv: 50.00%

### Lab2 (c) Finetune pruned model

In [9]:
def finetune_after_prune(net, trainloader, criterion, optimizer, prune=True):
    """
    Finetune the pruned model for a single epoch
    Make sure pruned weights are kept as zero
    """
    # Build a dictionary for the nonzero weights
    weight_mask = {}
    for name,layer in net.named_modules():
        if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
            # Your code here: generate a mask in GPU torch tensor to have 1 for nonzero element and 0 for zero element
            weight_mask[name] = (layer.weight != 0).float().to(device)

    global_steps = 0
    train_loss = 0
    correct = 0
    total = 0
    start = time.time()
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        if prune:
            for name,layer in net.named_modules():
                if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
                    # Your code here: Use weight_mask to make sure zero elements remains zero
                    layer.weight.data *= weight_mask[name]

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        global_steps += 1

        if global_steps % 50 == 0:
            end = time.time()
            batch_size = 256
            num_examples_per_second = 50 * batch_size / (end - start)
            print("[Step=%d]\tLoss=%.4f\tacc=%.4f\t%.1f examples/second"
                 % (global_steps, train_loss / (batch_idx + 1), (correct / total), num_examples_per_second))
            start = time.time()



In [28]:
# Get pruned model
net.load_state_dict(torch.load("pretrained_model.pt"))
for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        prune_by_percentage(layer, q=70.0)

# Training setup, do not change
batch_size=256
lr=0.002
reg=1e-4

print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=16)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.875, weight_decay=reg, nesterov=False)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# Model finetuning
for epoch in range(20):
    print('\nEpoch: %d' % epoch)
    net.train()
    finetune_after_prune(net, trainloader, criterion, optimizer,prune=True)
    #Start the testing code.
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    num_val_steps = len(testloader)
    val_acc = correct / total
    print("Test Loss=%.4f, Test acc=%.4f" % (test_loss / (num_val_steps), val_acc))

    if val_acc > best_acc:
        best_acc = val_acc
        print("Saving...")
        torch.save(net.state_dict(), "net_after_finetune.pt")


Epoch: 0
[Step=50]	Loss=0.4122	acc=0.8602	1151.1 examples/second
[Step=100]	Loss=0.3620	acc=0.8764	2643.2 examples/second
[Step=150]	Loss=0.3306	acc=0.8866	1678.5 examples/second
Test Loss=0.4251, Test acc=0.8682
Saving...

Epoch: 1
[Step=50]	Loss=0.2529	acc=0.9120	1179.3 examples/second
[Step=100]	Loss=0.2469	acc=0.9143	1514.4 examples/second
[Step=150]	Loss=0.2402	acc=0.9163	1803.8 examples/second
Test Loss=0.3958, Test acc=0.8759
Saving...

Epoch: 2
[Step=50]	Loss=0.2197	acc=0.9210	1172.0 examples/second
[Step=100]	Loss=0.2149	acc=0.9246	2384.1 examples/second
[Step=150]	Loss=0.2120	acc=0.9254	1749.0 examples/second
Test Loss=0.3800, Test acc=0.8800
Saving...

Epoch: 3
[Step=50]	Loss=0.1958	acc=0.9306	1148.7 examples/second
[Step=100]	Loss=0.1987	acc=0.9297	2347.9 examples/second
[Step=150]	Loss=0.1992	acc=0.9297	1909.6 examples/second
Test Loss=0.3705, Test acc=0.8840
Saving...

Epoch: 4
[Step=50]	Loss=0.1901	acc=0.9350	1190.4 examples/second
[Step=100]	Loss=0.1883	acc=0.9359	2543

In [29]:
# Check sparsity of the finetuned model, make sure it's not changed
net.load_state_dict(torch.load("net_after_finetune.pt"))

for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        # Your code here:
        # Convert the weight of "layer" to numpy array
        np_weight = layer.weight.data.cpu().numpy()
        # Count number of zeros
        zeros = np.sum(np_weight == 0)
        # Count number of parameters
        total = np_weight.size
        # Print sparsity
        print('Sparsity of '+name+': '+str(zeros/total))

test(net)

Sparsity of head_conv.0.conv: 0.6990740740740741
Sparsity of body_op.0.conv1.0.conv: 0.7000868055555556
Sparsity of body_op.0.conv2.0.conv: 0.7000868055555556
Sparsity of body_op.1.conv1.0.conv: 0.7000868055555556
Sparsity of body_op.1.conv2.0.conv: 0.7000868055555556
Sparsity of body_op.2.conv1.0.conv: 0.7000868055555556
Sparsity of body_op.2.conv2.0.conv: 0.7000868055555556
Sparsity of body_op.3.conv1.0.conv: 0.6998697916666666
Sparsity of body_op.3.conv2.0.conv: 0.6999782986111112
Sparsity of body_op.4.conv1.0.conv: 0.6999782986111112
Sparsity of body_op.4.conv2.0.conv: 0.6999782986111112
Sparsity of body_op.5.conv1.0.conv: 0.6999782986111112
Sparsity of body_op.5.conv2.0.conv: 0.6999782986111112
Sparsity of body_op.6.conv1.0.conv: 0.6999782986111112
Sparsity of body_op.6.conv2.0.conv: 0.7000054253472222
Sparsity of body_op.7.conv1.0.conv: 0.7000054253472222
Sparsity of body_op.7.conv2.0.conv: 0.7000054253472222
Sparsity of body_op.8.conv1.0.conv: 0.7000054253472222
Sparsity of body

### Lab2 (d) Iterative pruning

In [30]:
net.load_state_dict(torch.load("pretrained_model.pt"))
best_acc = 0.
for epoch in range(20):
    print('\nEpoch: %d' % epoch)

    net.train()
    if epoch<10:
        for name,layer in net.named_modules():
            if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
                # Increase model sparsity
                q = (epoch+1)*7
                prune_by_percentage(layer, q=q)
    if epoch<9:
        finetune_after_prune(net, trainloader, criterion, optimizer,prune=False)
    else:
        finetune_after_prune(net, trainloader, criterion, optimizer)

    #Start the testing code.
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    num_val_steps = len(testloader)
    val_acc = correct / total
    print("Test Loss=%.4f, Test acc=%.4f" % (test_loss / (num_val_steps), val_acc))

    if epoch>=10:
        if val_acc > best_acc:
            best_acc = val_acc
            print("Saving...")
            torch.save(net.state_dict(), "net_after_iterative_prune.pt")


Epoch: 0
[Step=50]	Loss=0.0472	acc=0.9854	1467.9 examples/second
[Step=100]	Loss=0.0471	acc=0.9852	2290.7 examples/second
[Step=150]	Loss=0.0481	acc=0.9845	2072.9 examples/second
Test Loss=0.3260, Test acc=0.9151

Epoch: 1
[Step=50]	Loss=0.0473	acc=0.9858	1272.5 examples/second
[Step=100]	Loss=0.0496	acc=0.9839	2673.8 examples/second
[Step=150]	Loss=0.0493	acc=0.9842	1936.9 examples/second
Test Loss=0.3261, Test acc=0.9154

Epoch: 2
[Step=50]	Loss=0.0502	acc=0.9841	1277.4 examples/second
[Step=100]	Loss=0.0490	acc=0.9843	2634.7 examples/second
[Step=150]	Loss=0.0509	acc=0.9839	2513.8 examples/second
Test Loss=0.3252, Test acc=0.9150

Epoch: 3
[Step=50]	Loss=0.0559	acc=0.9806	1441.3 examples/second
[Step=100]	Loss=0.0533	acc=0.9822	2072.6 examples/second
[Step=150]	Loss=0.0527	acc=0.9830	2613.5 examples/second
Test Loss=0.3291, Test acc=0.9135

Epoch: 4
[Step=50]	Loss=0.0611	acc=0.9807	1543.6 examples/second
[Step=100]	Loss=0.0622	acc=0.9799	1845.0 examples/second
[Step=150]	Loss=0.062

In [31]:
# Check sparsity of the final model, make sure it's 70%
net.load_state_dict(torch.load("net_after_iterative_prune.pt"))

for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        # Your code here: can copy from previous question
         # Convert the weight of "layer" to numpy array
        np_weight = layer.weight.data.cpu().numpy()
        # Count number of zeros
        zeros = np.sum(np_weight == 0)
        # Count number of parameters
        total = np_weight.size
        # Print sparsity
        print('Sparsity of '+name+': '+str(zeros/total))


test(net)

Sparsity of head_conv.0.conv: 0.6990740740740741
Sparsity of body_op.0.conv1.0.conv: 0.7000868055555556
Sparsity of body_op.0.conv2.0.conv: 0.7000868055555556
Sparsity of body_op.1.conv1.0.conv: 0.7000868055555556
Sparsity of body_op.1.conv2.0.conv: 0.7000868055555556
Sparsity of body_op.2.conv1.0.conv: 0.7000868055555556
Sparsity of body_op.2.conv2.0.conv: 0.7000868055555556
Sparsity of body_op.3.conv1.0.conv: 0.6998697916666666
Sparsity of body_op.3.conv2.0.conv: 0.6999782986111112
Sparsity of body_op.4.conv1.0.conv: 0.6999782986111112
Sparsity of body_op.4.conv2.0.conv: 0.6999782986111112
Sparsity of body_op.5.conv1.0.conv: 0.6999782986111112
Sparsity of body_op.5.conv2.0.conv: 0.6999782986111112
Sparsity of body_op.6.conv1.0.conv: 0.6999782986111112
Sparsity of body_op.6.conv2.0.conv: 0.7000054253472222
Sparsity of body_op.7.conv1.0.conv: 0.7000054253472222
Sparsity of body_op.7.conv2.0.conv: 0.7000054253472222
Sparsity of body_op.8.conv1.0.conv: 0.7000054253472222
Sparsity of body

### Lab2 (e) Global iterative pruning

In [10]:
def global_prune_by_percentage(net, q=70.0):
    """
    Pruning the weight paramters by threshold.
    :param q: pruning percentile. 'q' percent of the least
    significant weight parameters will be pruned.
    """
    # A list to gather all the weights
    flattened_weights = []
    # Find global pruning threshold
    for name,layer in net.named_modules():
        if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
            # Convert weight to numpy
            np_weight = layer.weight.data.cpu().numpy()

            # Flatten the weight and append to flattened_weights
            flattended_weight = np_weight.flatten()
            flattened_weights.append(flattended_weight)


    # Concate all weights into a np array
    flattened_weights = np.concatenate(flattened_weights)
    # Find global pruning threshold
    thres = np.percentile(np.abs(flattened_weights), q)

    # Apply pruning threshold to all layers
    for name,layer in net.named_modules():
        if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
            # Convert weight to numpy
            np_weight = layer.weight.data.cpu().numpy()

            # Generate a binary mask same shape as weight to decide which element to prune
            mask = (np.abs(np_weight) > thres).astype(np.float32)

            # Convert mask to torch tensor and put on GPU
            mask = torch.from_numpy(mask).to(device)

            # Multiply the weight by mask to perform pruning
            layer.weight.data *= mask


In [33]:
net.load_state_dict(torch.load("pretrained_model.pt"))
best_acc = 0.
for epoch in range(20):
    print('\nEpoch: %d' % epoch)
    q=(epoch+1)*7

    net.train()
    # Increase model sparsity
    if epoch<10:
        global_prune_by_percentage(net, q=q)
    if epoch<9:
        finetune_after_prune(net, trainloader, criterion, optimizer,prune=False)
    else:
        finetune_after_prune(net, trainloader, criterion, optimizer)

    #Start the testing code.
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    num_val_steps = len(testloader)
    val_acc = correct / total
    print("Test Loss=%.4f, Test acc=%.4f" % (test_loss / (num_val_steps), val_acc))

    if epoch>=10:
        if val_acc > best_acc:
            best_acc = val_acc
            print("Saving...")
            torch.save(net.state_dict(), "net_after_global_iterative_prune.pt")


Epoch: 0
[Step=50]	Loss=0.0481	acc=0.9852	1402.2 examples/second
[Step=100]	Loss=0.0477	acc=0.9855	2195.9 examples/second
[Step=150]	Loss=0.0469	acc=0.9861	2658.5 examples/second
Test Loss=0.3242, Test acc=0.9151

Epoch: 1
[Step=50]	Loss=0.0503	acc=0.9842	1537.9 examples/second
[Step=100]	Loss=0.0475	acc=0.9848	1834.4 examples/second
[Step=150]	Loss=0.0474	acc=0.9851	2621.5 examples/second
Test Loss=0.3267, Test acc=0.9155

Epoch: 2
[Step=50]	Loss=0.0480	acc=0.9845	1595.6 examples/second
[Step=100]	Loss=0.0476	acc=0.9854	1783.7 examples/second
[Step=150]	Loss=0.0486	acc=0.9847	2526.8 examples/second
Test Loss=0.3281, Test acc=0.9146

Epoch: 3
[Step=50]	Loss=0.0512	acc=0.9847	1464.0 examples/second
[Step=100]	Loss=0.0516	acc=0.9837	2130.5 examples/second
[Step=150]	Loss=0.0523	acc=0.9834	2091.4 examples/second
Test Loss=0.3310, Test acc=0.9128

Epoch: 4
[Step=50]	Loss=0.0522	acc=0.9823	1247.4 examples/second
[Step=100]	Loss=0.0524	acc=0.9827	2511.4 examples/second
[Step=150]	Loss=0.053

In [34]:
net.load_state_dict(torch.load("net_after_global_iterative_prune.pt"))

zeros_sum = 0
total_sum = 0
for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        # Your code here:
        # Convert the weight of "layer" to numpy array
        np_weight = layer.weight.data.cpu().numpy()
        # Count number of zeros
        zeros = np.sum(np_weight == 0)
        # Count number of parameters
        total = np_weight.size
        zeros_sum+=zeros
        total_sum+=total
        print('Sparsity of '+name+': '+str(zeros/total))
print('Total sparsity of: '+str(zeros_sum/total_sum))
test(net)

Sparsity of head_conv.0.conv: 0.24305555555555555
Sparsity of body_op.0.conv1.0.conv: 0.5503472222222222
Sparsity of body_op.0.conv2.0.conv: 0.5295138888888888
Sparsity of body_op.1.conv1.0.conv: 0.5186631944444444
Sparsity of body_op.1.conv2.0.conv: 0.5525173611111112
Sparsity of body_op.2.conv1.0.conv: 0.5186631944444444
Sparsity of body_op.2.conv2.0.conv: 0.5655381944444444
Sparsity of body_op.3.conv1.0.conv: 0.5251736111111112
Sparsity of body_op.3.conv2.0.conv: 0.5830078125
Sparsity of body_op.4.conv1.0.conv: 0.6159939236111112
Sparsity of body_op.4.conv2.0.conv: 0.6763237847222222
Sparsity of body_op.5.conv1.0.conv: 0.6111111111111112
Sparsity of body_op.5.conv2.0.conv: 0.7034505208333334
Sparsity of body_op.6.conv1.0.conv: 0.6154513888888888
Sparsity of body_op.6.conv2.0.conv: 0.6510959201388888
Sparsity of body_op.7.conv1.0.conv: 0.6623263888888888
Sparsity of body_op.7.conv2.0.conv: 0.718994140625
Sparsity of body_op.8.conv1.0.conv: 0.7478841145833334
Sparsity of body_op.8.con

### Lab 3 (b) and (c): Fixed-point quantization

In [30]:
Nbits = 6 #Change this value to finish (b) and (c)

net_6 = ResNetCIFAR(num_layers=20, Nbits=Nbits)
net_6 = net_6.to(device)
net_6.load_state_dict(torch.load("pretrained_model.pt"))
test(net_6)

Files already downloaded and verified
Test Loss=0.3364, Test accuracy=0.9145


In [12]:
Nbits = 5 #Change this value to finish (b) and (c)

net_5 = ResNetCIFAR(num_layers=20, Nbits=Nbits)
net_5 = net_5.to(device)
net_5.load_state_dict(torch.load("pretrained_model.pt"))
test(net_5)

Files already downloaded and verified
Test Loss=0.3390, Test accuracy=0.9112


In [13]:
Nbits = 4 #Change this value to finish (b) and (c)

net_4 = ResNetCIFAR(num_layers=20, Nbits=Nbits)
net_4 = net_4.to(device)
net_4.load_state_dict(torch.load("pretrained_model.pt"))
test(net_4)

Files already downloaded and verified
Test Loss=0.3861, Test accuracy=0.8972


In [14]:
Nbits = 3 #Change this value to finish (b) and (c)

net_3 = ResNetCIFAR(num_layers=20, Nbits=Nbits)
net_3 = net_3.to(device)
net_3.load_state_dict(torch.load("pretrained_model.pt"))
test(net_3)

Files already downloaded and verified
Test Loss=0.9874, Test accuracy=0.7662


In [15]:
Nbits = 2 #Change this value to finish (b) and (c)

net_2 = ResNetCIFAR(num_layers=20, Nbits=Nbits)
net_2 = net_2.to(device)
net_2.load_state_dict(torch.load("pretrained_model.pt"))
test(net_2)

Files already downloaded and verified
Test Loss=9.5441, Test accuracy=0.0899


In [14]:
# Quantized model finetuning
finetune(net_4, epochs=20, batch_size=256, lr=0.002, reg=1e-4)

# Load the model with best accuracy
net_4.load_state_dict(torch.load("quantized_net_after_finetune.pt"))
test(net_4)

==> Preparing data..
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Files already downloaded and verified

Epoch: 0
[Step=50]	Loss=0.0682	acc=0.9778	1131.3 examples/second
[Step=100]	Loss=0.0642	acc=0.9790	2329.1 examples/second
[Step=150]	Loss=0.0655	acc=0.9781	1698.5 examples/second
Test Loss=0.3342, Test acc=0.9090
Saving...

Epoch: 1
[Step=200]	Loss=0.0630	acc=0.9795	917.7 examples/second
[Step=250]	Loss=0.0626	acc=0.9800	2080.9 examples/second
[Step=300]	Loss=0.0643	acc=0.9790	2039.2 examples/second
[Step=350]	Loss=0.0627	acc=0.9793	1755.5 examples/second
Test Loss=0.3360, Test acc=0.9110
Saving...

Epoch: 2
[Step=400]	Loss=0.0494	acc=0.9824	979.1 examples/second
[Step=450]	Loss=0.0619	acc=0.9784	2266.8 examples/second
[Step=500]	Loss=0.0598	acc=0.9792	1713.2 examples/second
[Step=550]	Loss=0.0603	acc=0.9796	2124.4 examples/second
Test Loss=0.3341, Test acc=0.9106

Epoch: 3
[Step=600]	Loss=0.0518	acc=0.9840	981.2 examples/second
[Step=650]	Loss=0.0587	acc=0.9812	2272.8 examples/second
[Step=700]	Loss=0.0604	acc=0.9801	1652.3 examples/second
[Step=

In [16]:
# Quantized model finetuning
finetune(net_3, epochs=20, batch_size=256, lr=0.002, reg=1e-4)

# Load the model with best accuracy
net_3.load_state_dict(torch.load("quantized_net_after_finetune.pt"))
test(net_3)

==> Preparing data..
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Files already downloaded and verified

Epoch: 0
[Step=50]	Loss=0.1657	acc=0.9393	1299.6 examples/second
[Step=100]	Loss=0.1573	acc=0.9437	1935.2 examples/second
[Step=150]	Loss=0.1484	acc=0.9474	2486.3 examples/second
Test Loss=0.3991, Test acc=0.8948
Saving...

Epoch: 1
[Step=200]	Loss=0.1444	acc=0.9502	1011.6 examples/second
[Step=250]	Loss=0.1339	acc=0.9514	1857.6 examples/second
[Step=300]	Loss=0.1280	acc=0.9535	2084.4 examples/second
[Step=350]	Loss=0.1247	acc=0.9550	2416.3 examples/second
Test Loss=0.3868, Test acc=0.8977
Saving...

Epoch: 2
[Step=400]	Loss=0.1101	acc=0.9580	1079.4 examples/second
[Step=450]	Loss=0.1137	acc=0.9581	1999.2 examples/second
[Step=500]	Loss=0.1118	acc=0.9597	2038.3 examples/second
[Step=550]	Loss=0.1143	acc=0.9592	2418.3 examples/second
Test Loss=0.3725, Test acc=0.8995
Saving...

Epoch: 3
[Step=600]	Loss=0.1050	acc=0.9613	1059.2 examples/second
[Step=650]	Loss=0.1086	acc=0.9621	1937.0 examples/second
[Step=700]	Loss=0.1059	acc=0.9625	2132.0 examples/

In [17]:
# Quantized model finetuning
finetune(net_2, epochs=20, batch_size=256, lr=0.002, reg=1e-4)

# Load the model with best accuracy
net_2.load_state_dict(torch.load("quantized_net_after_finetune.pt"))
test(net_2)

==> Preparing data..
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Files already downloaded and verified

Epoch: 0
[Step=50]	Loss=1.3754	acc=0.6118	1139.5 examples/second
[Step=100]	Loss=1.0772	acc=0.6816	2348.6 examples/second
[Step=150]	Loss=0.9408	acc=0.7145	1707.2 examples/second
Test Loss=0.7622, Test acc=0.7730
Saving...

Epoch: 1
[Step=200]	Loss=0.5926	acc=0.7998	1024.2 examples/second
[Step=250]	Loss=0.5690	acc=0.8078	1838.6 examples/second
[Step=300]	Loss=0.5601	acc=0.8114	2264.3 examples/second
[Step=350]	Loss=0.5425	acc=0.8157	1741.9 examples/second
Test Loss=0.6558, Test acc=0.7991
Saving...

Epoch: 2
[Step=400]	Loss=0.4834	acc=0.8306	932.0 examples/second
[Step=450]	Loss=0.4752	acc=0.8339	2079.8 examples/second
[Step=500]	Loss=0.4716	acc=0.8365	2361.6 examples/second
[Step=550]	Loss=0.4629	acc=0.8402	1669.0 examples/second
Test Loss=0.5729, Test acc=0.8211
Saving...

Epoch: 3
[Step=600]	Loss=0.4331	acc=0.8512	936.9 examples/second
[Step=650]	Loss=0.4241	acc=0.8532	2199.1 examples/second
[Step=700]	Loss=0.4298	acc=0.8522	2265.1 examples/se

### Lab3 (d) Quantize pruned model

In [19]:
# Define quantized model and load weight
Nbits = 4 #Change this value to finish (d)

net_4B = ResNetCIFAR(num_layers=20, Nbits=Nbits)
net_4B= net_4B.to(device)
net_4B.load_state_dict(torch.load("net_after_global_iterative_prune.pt"))
test(net_4B)


Files already downloaded and verified
Test Loss=0.4115, Test accuracy=0.8722


In [20]:
# Quantized model finetuning
finetune(net_4B, epochs=20, batch_size=256, lr=0.002, reg=1e-4)

# Load the model with best accuracy
net_4B.load_state_dict(torch.load("quantized_net_after_finetune.pt"))
test(net_4B)

==> Preparing data..
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Files already downloaded and verified

Epoch: 0
[Step=50]	Loss=0.7270	acc=0.7520	1132.4 examples/second
[Step=100]	Loss=0.5061	acc=0.8271	2347.7 examples/second
[Step=150]	Loss=0.4209	acc=0.8557	1685.8 examples/second
Test Loss=0.3579, Test acc=0.8808
Saving...

Epoch: 1
[Step=200]	Loss=0.2137	acc=0.9268	954.7 examples/second
[Step=250]	Loss=0.2148	acc=0.9256	1991.2 examples/second
[Step=300]	Loss=0.2090	acc=0.9282	2109.4 examples/second
[Step=350]	Loss=0.2042	acc=0.9298	1783.8 examples/second
Test Loss=0.3414, Test acc=0.8889
Saving...

Epoch: 2
[Step=400]	Loss=0.2014	acc=0.9243	953.8 examples/second
[Step=450]	Loss=0.1851	acc=0.9351	2376.3 examples/second
[Step=500]	Loss=0.1850	acc=0.9360	1996.7 examples/second
[Step=550]	Loss=0.1838	acc=0.9364	1940.3 examples/second
Test Loss=0.3288, Test acc=0.8940
Saving...

Epoch: 3
[Step=600]	Loss=0.1573	acc=0.9463	976.6 examples/second
[Step=650]	Loss=0.1695	acc=0.9412	2341.7 examples/second
[Step=700]	Loss=0.1720	acc=0.9410	1739.8 examples/sec

In [21]:
# Define quantized model and load weight
Nbits = 3 #Change this value to finish (d)

net_3B = ResNetCIFAR(num_layers=20, Nbits=Nbits)
net_3B= net_3B.to(device)
net_3B.load_state_dict(torch.load("net_after_global_iterative_prune.pt"))
test(net_3B)


Files already downloaded and verified
Test Loss=1.1141, Test accuracy=0.6331


In [22]:
# Quantized model finetuning
finetune(net_3B, epochs=20, batch_size=256, lr=0.002, reg=1e-4)

# Load the model with best accuracy
net_3B.load_state_dict(torch.load("quantized_net_after_finetune.pt"))
test(net_3B)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified

Epoch: 0
[Step=50]	Loss=1.4114	acc=0.5343	1191.0 examples/second
[Step=100]	Loss=1.1732	acc=0.6066	2322.3 examples/second
[Step=150]	Loss=1.0196	acc=0.6564	2337.4 examples/second
Test Loss=0.6304, Test acc=0.7863
Saving...

Epoch: 1
[Step=200]	Loss=0.5586	acc=0.8027	1084.2 examples/second
[Step=250]	Loss=0.5164	acc=0.8226	1673.1 examples/second
[Step=300]	Loss=0.4947	acc=0.8310	2268.7 examples/second
[Step=350]	Loss=0.4795	acc=0.8362	2382.3 examples/second
Test Loss=0.5143, Test acc=0.8232
Saving...

Epoch: 2
[Step=400]	Loss=0.4100	acc=0.8652	974.8 examples/second
[Step=450]	Loss=0.4143	acc=0.8584	1755.8 examples/second
[Step=500]	Loss=0.4068	acc=0.8591	2226.9 examples/second
[Step=550]	Loss=0.3970	acc=0.8621	2243.3 examples/second
Test Loss=0.4785, Test acc=0.8419
Saving...

Epoch: 3
[Step=600]	Loss=0.3470	acc=0.8773	1067.1 examples/second
[Step=650]	Loss=0.3470	acc=0.8793	1814.1 examples

In [23]:
# Define quantized model and load weight
Nbits = 2 #Change this value to finish (d)

net_2B = ResNetCIFAR(num_layers=20, Nbits=Nbits)
net_2B= net_2B.to(device)
net_2B.load_state_dict(torch.load("net_after_global_iterative_prune.pt"))
test(net_2B)

Files already downloaded and verified
Test Loss=7358.1566, Test accuracy=0.1000


In [24]:
# Quantized model finetuning
finetune(net_2B, epochs=20, batch_size=256, lr=0.002, reg=1e-4)

# Load the model with best accuracy
net_2B.load_state_dict(torch.load("quantized_net_after_finetune.pt"))
test(net_2B)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified

Epoch: 0
[Step=50]	Loss=2.2657	acc=0.1443	1161.9 examples/second
[Step=100]	Loss=2.2269	acc=0.1528	2233.3 examples/second
[Step=150]	Loss=2.2026	acc=0.1613	1692.3 examples/second
Test Loss=2.1293, Test acc=0.1851
Saving...

Epoch: 1
[Step=200]	Loss=2.1619	acc=0.1895	1037.5 examples/second
[Step=250]	Loss=2.1256	acc=0.1945	1897.9 examples/second
[Step=300]	Loss=2.1206	acc=0.2029	2195.4 examples/second
[Step=350]	Loss=2.1141	acc=0.2072	1714.3 examples/second
Test Loss=2.1030, Test acc=0.2227
Saving...

Epoch: 2
[Step=400]	Loss=2.0908	acc=0.2212	907.9 examples/second
[Step=450]	Loss=2.0843	acc=0.2302	2320.9 examples/second
[Step=500]	Loss=2.0780	acc=0.2317	1841.8 examples/second
[Step=550]	Loss=2.0735	acc=0.2346	1999.6 examples/second
Test Loss=2.0748, Test acc=0.2330
Saving...

Epoch: 3
[Step=600]	Loss=2.0606	acc=0.2461	978.6 examples/second
[Step=650]	Loss=2.0484	acc=0.2471	2377.8 examples/

### Lab3 (e) Symmetric quantization
#### Implement symmetric quantization in FP_layers.py, and repeat the process in (b)

In [5]:
Nbits = 6 #Change this value to finish (b) and (c)


net = ResNetCIFAR(num_layers=20, Nbits=Nbits,symmetric=True)
net = net.to(device)
net.load_state_dict(torch.load("pretrained_model.pt"))
test(net)

Files already downloaded and verified
Test Loss=0.3276, Test accuracy=0.9124


In [6]:
Nbits = 5 #Change this value to finish (b) and (c)


net = ResNetCIFAR(num_layers=20, Nbits=Nbits,symmetric=True)
net = net.to(device)
net.load_state_dict(torch.load("pretrained_model.pt"))
test(net)

Files already downloaded and verified
Test Loss=0.3520, Test accuracy=0.9083


In [7]:
Nbits = 4 #Change this value to finish (b) and (c)


net = ResNetCIFAR(num_layers=20, Nbits=Nbits,symmetric=True)
net = net.to(device)
net.load_state_dict(torch.load("pretrained_model.pt"))
test(net)

Files already downloaded and verified
Test Loss=0.4227, Test accuracy=0.8875


In [8]:
Nbits = 3#Change this value to finish (b) and (c)


net = ResNetCIFAR(num_layers=20, Nbits=Nbits,symmetric=True)
net = net.to(device)
net.load_state_dict(torch.load("pretrained_model.pt"))
test(net)

Files already downloaded and verified
Test Loss=2.3739, Test accuracy=0.5185


In [9]:
Nbits = 2 #Change this value to finish (b) and (c)


net = ResNetCIFAR(num_layers=20, Nbits=Nbits,symmetric=True)
net = net.to(device)
net.load_state_dict(torch.load("pretrained_model.pt"))
test(net)

Files already downloaded and verified
Test Loss=42.7781, Test accuracy=0.1000
